In [2]:
import pandas as pd
import re
import textstat
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
df = pd.read_csv('full_gpt_2.csv')
df.info()

In [ ]:
df = df.iloc[:,1:]
df.dropna(inplace=True)
df.head()

In [5]:
df['Status'].unique()

array(['Wrong Answer', 'Run Time Error', 'Accepted',
       'Time Limit Exceeded', 'Memory Limit Exceeded'], dtype=object)

# New features

In [6]:
# the Flesch Reading Ease Score
df['FRE'] = df['Description'].apply(lambda x: textstat.flesch_reading_ease(x))
# Sentence Count
df['SenCount'] = df['Description'].apply(lambda x: textstat.sentence_count(x))
# Lexicon Count
df['LexCount'] = df['Description'].apply(lambda x: textstat.lexicon_count(x))
# Readability Consensus based upon all the above tests (text standard)
df['textStd'] = df['Description'].apply(lambda x: textstat.text_standard(x, float_output=True))

df_len = len(df)
for index in range(df_len):
    if df['Status'][index] != 'Accepted':
        df['Pass'][index] = df['Pass'][index] - 1
# Test_diff = Total - Pass
df['Test_diff'] = df['Pass']/df['Total']

In [7]:
print(df_len)

3323


In [ ]:
df.head()

In [ ]:
df.describe()

# Difficulty 

In [10]:
sns.set(context="notebook", style="darkgrid", palette="bright")

In [ ]:
g = sns.displot(data=df, x="Difficulty", kde=True, bins=10, binrange=(1,10), height=5, aspect=1)
#g.set(title ='Histogram of Difficulty - 10 bins')
g.savefig("fig/hist_diff.png")

# Status

In [ ]:
g = sns.displot(data=df, x="Status", height=5, aspect=1)
g.set(title ='Histogram of Status')

# Difficulty + Status

In [ ]:
g = sns.displot(data=df, x="Difficulty", hue='Status', multiple="stack", bins=5, binrange=(0,10), height=5, aspect=1)
g.savefig("fig/hist_diff_status.png")

In [ ]:
sns.displot(data=df, x="textStd", hue='Status', multiple="stack", bins=12, binrange=(6,19), height=5, aspect=1)

In [ ]:
sns.displot(data=df, x="SenCount", hue='Status', multiple="stack", bins=10, binrange=(25,85), height=5, aspect=1)

In [ ]:
sns.displot(data=df, x="SenCount", hue='Status', multiple="stack", bins=12, binrange=(2,39), height=5, aspect=1)

In [ ]:
g = sns.displot(data=df, x="Difficulty", hue='Status', kind='kde', height=5, aspect=1)
g.savefig("fig/dens_diff.png")

In [ ]:
df['Status'].unique()

In [ ]:
import numpy as np
import scipy

def ks_analysis(s): 
    wrong_answer = []
    accepted = []
    error = []
    timelimit = []
    memlimit = []
    print(s)
    for index in df.index:
        if(df["Status"][index] == "Wrong Answer"):
            wrong_answer.append(df[s][index])        
        elif(df["Status"][index] == "Accepted"):
            accepted.append(df[s][index])    
        elif(df["Status"][index] == "Run Time Error"):
            error.append(df[s][index])
        elif(df["Status"][index] == "Time Limit Exceeded"):
            timelimit.append(df[s][index])
        elif(df["Status"][index] == "Memory Limit Exceeded"):
            memlimit.append(df[s][index])
    print("Accepted vs Wrong Answer &", scipy.stats.kstest(accepted, wrong_answer).pvalue)
    print("Accepted vs Run Time Error &", scipy.stats.kstest(accepted, error).pvalue)
    print("Accepted vs Time Limit Exceeded &", scipy.stats.kstest(accepted, timelimit).pvalue)
    print("Accepted vs Memory Limit Exceeded &", scipy.stats.kstest(accepted, memlimit).pvalue)
    print("Wrong Answer vs Error &", scipy.stats.kstest(wrong_answer, error).pvalue)
    print("Wrong Answer vs Time Limit Exceeded &", scipy.stats.kstest(wrong_answer, timelimit).pvalue)
    print("Wrong Answer vs Memory Limit Exceeded &", scipy.stats.kstest(wrong_answer, memlimit).pvalue)
    print("Run Time Error vs Time Limit Exceeded &", scipy.stats.kstest(error, timelimit).pvalue)
    print("Run Time Error vs Memory Limit Exceeded &", scipy.stats.kstest(error, memlimit).pvalue)
    print("Time Limit Exceeded vs Memory Limit Exceeded &", scipy.stats.kstest(timelimit, memlimit).pvalue)
    sns.displot(data=df, x=s, hue='Status', kind='kde', height=5, aspect=1)
    print()

#measures = ['Difficulty','FRE','LexCount','textStd']
measures = ['Difficulty']
for m in measures:
    ks_analysis(m)

In [20]:
import numpy as np
import scipy

def status_range(mindiff, maxdiff): 
    #print('Difficulty range:', mindiff, maxdiff)
    print("\\multicolumn{6}{c}{\\textbf{Problem Difficulty: %1.1f - %1.1f}} \\\\" % (mindiff,maxdiff))
    print("\\hline")
    prefix_str = """\\textbf{Status} & \\multicolumn{1}{l}{\\textbf{Count}} & \\multicolumn{1}{l}{\\textbf{Mean}} & \\multicolumn{1}{l}{\\textbf{Minimum}} & \\multicolumn{1}{l}{\\textbf{Maximum}} & \\multicolumn{1}{l}{\\textbf{Std. Dev.}} \\\\"""
    print(prefix_str)
    df_tmp = df[(df['Difficulty'] >= mindiff) & (df['Difficulty'] < maxdiff)]
    #print(df_tmp)
    #print('Problem count:', len(df_tmp))
    status = ['Accepted','Wrong Answer', 'Run Time Error', 'Time Limit Exceeded', 'Memory Limit Exceeded']
    for st in status:
        #print(st)
        df_stat = df_tmp[df_tmp['Status'] == st]
        #print(df_stat)
        if (st == 'Accepted'):
            if len(df_stat['Test_diff']) > 0:
                x = df_stat['Test_diff'].describe(percentiles=[])
                print("\\textit{%s} & %d &  &  &  & \\\\" % (st,len(df_stat)))
            else:
                print("\\textit{%s} & 0 &  &  &  &   \\\\" % (st))
        else:
            if len(df_stat['Test_diff']) > 0:
                x = df_stat['Test_diff'].describe(percentiles=[])
                print("\\textit{%s} & %d & %2.3f & %2.3f & %2.3f & %2.3f \\\\" % (st,len(df_stat),x['mean'],x['min'],x['max'],x['std']))
            else:
                print("\\textit{%s} & 0 &  &  &  &   \\\\" % (st))
    print("Total & %d &  &  &  &   \\\\" % len(df_tmp))
    print("\\hline\\hline")

In [21]:
import numpy as np
import scipy

def status_range(mindiff, maxdiff): 
    #print('Difficulty range:', mindiff, maxdiff)
    print("\\multicolumn{6}{c}{\\textbf{Problem Difficulty: %1.1f - %1.1f}} \\\\" % (mindiff,maxdiff))
    print("\\hline")
    prefix_str = """\\textbf{Status} & \\multicolumn{1}{l}{\\textbf{Count}} & \\multicolumn{1}{l}{\\textbf{Median}} & \\multicolumn{1}{l}{\\textbf{Mean}} & \\multicolumn{1}{l}{\\textbf{Min}} & \\multicolumn{1}{l}{\\textbf{Max}} \\\\"""
    print(prefix_str)
    df_tmp = df[(df['Difficulty'] >= mindiff) & (df['Difficulty'] < maxdiff)]
    #print(df_tmp)
    #print('Problem count:', len(df_tmp))
    status = ['Accepted','Wrong Answer', 'Run Time Error', 'Time Limit Exceeded', 'Memory Limit Exceeded']
    for st in status:
        #print(st)
        df_stat = df_tmp[df_tmp['Status'] == st]
        x = df_stat['Total'].describe(percentiles=[])
        print("\\textit{%s} & %d & %2.1f & %2.1f & %2.1f & %2.1f \\\\" % (st,len(df_stat),x['50%'],x['mean'],x['min'],x['max']))
        #    else:
        #        print("\\textit{%s} & 0 &  &  &  &   \\\\" % (st))
    print("Total & %d &  &  &  &   \\\\" % len(df_tmp))
    print("\\hline\\hline")

In [ ]:
df['Test_diff'].describe(percentiles=[])['50%']

In [ ]:
df['Total'].unique().mean()#.describe()

In [ ]:
status_range(0,2)
status_range(2,4)
status_range(4,6)
status_range(6,8)
status_range(8,10)

In [ ]:
g = sns.FacetGrid(df, col="Status", col_wrap=2)
g.map_dataframe(sns.histplot, x="Difficulty", binrange=(0,10),binwidth = 2)
g.savefig("fig/hist_diff_status_plots.png")

# Correlation

In [ ]:
df.info()

In [ ]:
from sklearn import preprocessing

df_encode = df.copy()
df_encode = df_encode.drop(['Problem','Description'], axis=1)
le = preprocessing.LabelEncoder()
df_encode['Status'] = le.fit_transform(df_encode['Status'])

sns.set(font_scale = 2)
plt.figure(figsize=(20,20))
sns.heatmap(df_encode.corr(), cmap="YlGnBu", annot=True)

# Test_diff

In [ ]:
sns.set(font_scale = 1)
sns.scatterplot(data=df, x='Difficulty', y='Test_diff')

In [ ]:
sns.scatterplot(data=df, x='Difficulty', y='Test_diff', hue='Status')

In [ ]:
sns.jointplot(data=df, x='Difficulty', y='Test_diff')

In [ ]:
sns.jointplot(data=df, x='Difficulty', y='Test_diff', hue='Status', height=7)